In [1]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news LIMIT 1000")
display(df)

StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, b519eaa1-9061-4710-8c43-63379d49945e)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 4, Finished, Available)

In [3]:
# Import the model and configure the input and output columns

model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error")
        )


StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 5, Finished, Available)

In [4]:
# Apply the model to our dataframe

result = model.transform(df)

StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 6, Finished, Available)

In [5]:
display(result)

StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 335fefad-749b-4d9f-a41d-56bfa1272e0f)

In [6]:
# Create Sentiment Column

from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment",col("response.documents.sentiment"))

StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 8, Finished, Available)

In [7]:
display(sentiment_df)

StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 00923571-795d-41ba-8f87-3c5af46c8135)

In [8]:
sentiment_df_final = sentiment_df.drop("error", "response")

StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 10, Finished, Available)

In [9]:
display(sentiment_df_final)

StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6ca068db-88c3-4a73-b98b-2c0f9b632859)

In [16]:
from pyspark.sql.functions import col, to_date

sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MM-yyyy"))

StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 18, Finished, Available)

# Type 1 Merge

In [10]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'

    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_sentiment_df_final source_view

                    ON source_view.url = target_table.url

                    WHEN MATCHED AND
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *

    """)

StatementMeta(, 24c00743-a626-408a-9d37-fa9fd52a5b3b, 12, Finished, Available)

Table Already Exists
